In [ ]:
def debug_pdf_data(content, filename):
    """Debug helper to verify PDF data"""
    print(f"\nProcessing {filename}")
    if not content:
        print("No content received")
        return False
        
    if not content.startswith('data:'):
        print("Content missing data URI prefix")
        return False
        
    parts = content.split(',')
    if len(parts) != 2:
        print("Invalid data URI format")
        return False
        
    header, b64_content = parts
    if 'application/pdf' not in header:
        print(f"Unexpected content type: {header}")
        
    try:
        decoded = base64.b64decode(b64_content)
        print(f"Successfully decoded {len(decoded)} bytes")
        return True
    except Exception as e:
        print(f"Base64 decode failed: {e}")
        return False

def register_file_upload_callback(app, name):
    @app.callback(
        Output(f'uploaded-file-{name}', component_property='data'),
        Input(f'upload-data-{name}', component_property='contents'),
        State(f'upload-data-{name}', component_property='filename'),
    )
    def upload_files(contents, filenames):
        if contents is None:
            return [], [], []
            
        if not isinstance(contents, list):
            contents = [contents]
            filenames = [filenames]
            
        print(f"Processing {len(contents)} files")
        
        encoded_files = []
        file_contents = []
        file_names = []
        
        for content, filename in zip(contents, filenames):
            try:
                if 'pdf' in filename.lower():
                    # Validate and process PDF
                    if debug_pdf_data(content, filename):
                        encoded_files.append(content)  # Keep original data URI
                        
                        # Extract text content
                        try:
                            content_type, content_string = content.split(',')
                            decoded = base64.b64decode(content_string)
                            
                            # Try PyPDF2 first
                            text_content = []
                            pdf = io.BytesIO(decoded)
                            reader = PdfReader(pdf)
                            
                            for page in reader.pages:
                                text = page.extract_text()
                                if text:
                                    text_content.append(text)
                                    
                            if not text_content:  # If no text was extracted, try pdfplumber
                                pdf.seek(0)
                                with pdfplumber.open(pdf) as plumber_pdf:
                                    for page in plumber_pdf.pages:
                                        text = page.extract_text()
                                        if text:
                                            text_content.append(text)
                            
                            file_contents.append('\n'.join(text_content))
                            file_names.append(filename)
                            print(f"Successfully processed PDF: {filename}")
                            
                        except Exception as e:
                            print(f"Error extracting PDF text: {e}")
                            file_contents.append("")  # Add empty content
                            file_names.append(filename)
                            
                elif 'txt' in filename.lower():
                    try:
                        content_type, content_string = content.split(',')
                        decoded = base64.b64decode(content_string)
                        text_content = decoded.decode('utf-8')
                        
                        encoded_files.append(content)
                        file_contents.append(text_content)
                        file_names.append(filename)
                        print(f"Successfully processed text file: {filename}")
                        
                    except Exception as e:
                        print(f"Error processing text file: {e}")
                        continue
                        
            except Exception as e:
                print(f"Error processing file {filename}: {e}")
                continue
                
        print(f"Processed {len(encoded_files)} files successfully")
        return encoded_files, file_contents, file_names

def register_update_tabs_callback(app, name):
    @app.callback(
        Output('file-tabs' + name, component_property='tabs'),
        Output('file-tabs' + name, component_property='children'),
        Input('uploaded-file' + name, component_property='data'),
        Input('file-tabs' + name, component_property='active_tab'),
    )
    def update_tabs(data, active_tab):
        if not data or not data[0]:
            return [], []
            
        encoded_files, file_contents, file_names = data
        
        # Create tabs and containers
        tabs = []
        children = []
        
        for i, (file_name, encoded_file) in enumerate(zip(file_names, encoded_files)):
            tab_id = f'tab{i + 1}{name}'
            
            # Create tab
            tabs.append({
                'id': tab_id,
                'title': file_name
            })
            
            # Create container for each tab content
            tab_content = html.Div(
                [
                    # PDF viewer
                    html.Div(
                        html.Iframe(
                            id=f'pdf-viewer-{tab_id}',
                            src=encoded_file,
                            style={
                                'width': '100%',
                                'height': '800px',
                                'border': 'none',
                                'display': 'block',
                            }
                        ),
                        style={
                            'width': '100%',
                            'marginBottom': '20px',
                        }
                    ),
                ],
                id=f'tab-content-{tab_id}',
                style={
                    'width': '100%',
                    'height': 'auto',
                    'overflow': 'hidden',
                    'padding': '20px',
                    'backgroundColor': '#ffffff',
                }
            )
            
            children.append(tab_content)
        
        return tabs, children

def register_update_display_callback(app, name):
    @app.callback(
        Output("display-conversation" + name, component_property="children"),
        [Input("store-conversation" + name, component_property="data")]
    )
    def update_display(chat_history):
        if not chat_history:
            return html.Div(
                html.Img(src=app.get_asset_url('Logo.png'), 
                        style={'display': 'flex',
                               'justifyContent': 'center',
                               'alignItems': 'center',
                               'height': '200px',
                               'width': '200px'}),
                style={'display': 'flex', 'justifyContent': 'center', 'alignItems': 'center', 'height': '80vh'}
            )
            
        return [
            textbox(x, box="user") if i % 2 == 0 else textbox(x, box="AI")
            for i, x in enumerate(chat_history.split("<split>")[:-1])
        ]

def register_clear_input_callback(app, name):
    @app.callback(
        Output("chat-input" + name, component_property="value"),
        [Input("submit" + name, component_property="n_clicks"),
         Input("chat-input" + name, component_property="n_submit")],
    )
    def clear_input(n_clicks, n_submit):
        return ""